# HD Parent Key Exposure: Example (c++)

<br>
<img src="images/hd_parent_exposure.jpg" alt="drawing" style="" width="700px"/>



### Libbitcoin-System and secp256k1 setup

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 

// secp256k1 supporting files.
#include "supporting_scripts/secp256k1_initializer.hpp"

### Derive parent XPRV from parent XPUB and child private key.

In [19]:
{
    // Generate m/44'/0'/0'/0 parent key from entropy.
    // ---------------------------------------------------------------------------
    
    bc::data_chunk my_entropy_128(16);
    bc::pseudo_random_fill(my_entropy_128);

    bc::wallet::hd_private m(my_entropy_128, bc::wallet::hd_private::mainnet);
    auto m_44h_0h_0h_0 = m.derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(0 + bc::wallet::hd_first_hardened_key)
        .derive_private(0 + bc::wallet::hd_first_hardened_key)
        .derive_private(0);

    std::cout << "Parent Private Key: "
              << bc::encode_base16(bc::to_chunk(m_44h_0h_0h_0.secret())) 
              << std::endl;

    // 1) M/44'/0'/0'/0 and m/44'/0'/0'/0/1 secret are exposed.
    // ---------------------------------------------------------------------------
    uint32_t child_index(1);
    
    // Parent XPUB exposed.
    auto M_44h_0h_0h_0 = m_44h_0h_0h_0.to_public();
    
    // Child XPRV secret exposed.
    auto m_44h_0h_0h_0_1_secret = m_44h_0h_0h_0.derive_private(child_index).secret();

    // 2) Derive L256 from hmac_sha512_hash(parent public key||index, chaincode).
    // ---------------------------------------------------------------------------
    auto data = bc::splice(M_44h_0h_0h_0.point(), bc::to_big_endian(child_index));
    auto my_byte_array_parts = bc::split(bc::hmac_sha512_hash(data, M_44h_0h_0h_0.chain_code()));
    auto left_256 = my_byte_array_parts.left;

    // 3) Parent Private(m) = child private(m_0) - L256.
    // ---------------------------------------------------------------------------
    
    const auto context = bc::verification.context();
    if (secp256k1_ec_privkey_negate(context,left_256.data()) != 1)
        return 1;
    bc::ec_add(left_256, m_44h_0h_0h_0_1_secret);

    // Print out computed parent private key.
    // ---------------------------------------------------------------------------
    std::cout << "Computed Private Parent Key: "
            << bc::encode_base16(bc::to_chunk(left_256)) << std::endl;
}

Parent Private Key: 80e987ba2c354c450168dde679760f2df343d9690a0e503f20981e04e499f9ac
Computed Private Parent Key: 80e987ba2c354c450168dde679760f2df343d9690a0e503f20981e04e499f9ac
